In [ ]:
import os
from Headlines_Classifier import *
from helpFunctions import *
pd.options.mode.chained_assignment = None
import torch
import os
from helpFunctions import *
import pandas as pd
import torch
import time
from keras.layers import *
from helpFunctions import *
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import logging

logging.set_verbosity_error()
import re


def remove_symbols(text):
    # Define regular expression pattern to match awkward symbols
    symbol_pattern = r'[^\w\s\.\?\!\',;:\-]'
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove symbols from text using regex
    text = re.sub(symbol_pattern, '', text)
    
    return text


def classifyData(start, end, ticker, classifier = 'ProsusAI/finbert'):
    news = getLatestData('headlines', ticker)
    print(len(news))
    if ticker is None or ticker == "marketsNews":
        news['Date'] = news['Date'] + " " + news['Time']
    news['Date'] =  pd.to_datetime(news['Date'], infer_datetime_format=True)
    news = news.sort_values(by = 'Date')
    news_rng = news[(news['Date'] > start) & (news['Date'] < end)]
    news_rng.loc[:, 'Headline'] = news_rng['Headline'].apply(remove_symbols)


    df_total = pd.DataFrame(columns = ["Headline", "Datetime", "Positive", "Negative", "Neutral"])

    if classifier == "GPT":
        #openai.api_key = API_KEY

        str_list = []
        for index, row in news_rng.iterrows():
            news = row['Headline']
            prompt= "Decide whether a News sentiment is positive, neutral, or negative.\n\nNews: \"%s\"\nSentiment: " %news
            str_list.append(prompt)

        limit = 20
        blocks = int(len(str_list) / limit)

        positive = []
        negative = []
        neutral = []

        table = {"Headline": news_rng['Headline'],
            "Datetime": news_rng['Date'],
            "Positive":positive,
            "Negative":negative, 
            "Neutral":neutral
            }

        df = pd.DataFrame(table, columns = ["Headline", "Datetime", "Positive", "Negative", "Neutral"])
        df_total = df_total.append(df, ignore_index = True)

    else:
        tokenizer = AutoTokenizer.from_pretrained(classifier)
        model = AutoModelForSequenceClassification.from_pretrained(classifier)

        block_size = 250
        blocks = int(len(news_rng) / block_size)
        if blocks < 1:
            blocks = 1
        for i in range(0, blocks):
            df = ""
            print('Run: ' + str(i+1) + '/' + str(blocks))
            print("-" * 60)
            if i == blocks-1:
                #headlines = news_rng['Headline'][i*block_size: ].to_list()
                #dates = news_rng['Date'][i*block_size:].to_list()
                headlines = news_rng['Headline'].iloc[i*block_size: ].to_list()
                dates = news_rng['Date'].iloc[i*block_size:].to_list()
            else:
                #headlines = news_rng['Headline'][i*block_size: (i+1) * block_size].to_list()
                #dates = news_rng['Date'][i*block_size: (i+1)* block_size].to_list()
                headlines = news_rng['Headline'].iloc[i*block_size: (i+1) * block_size].to_list()
                dates = news_rng['Date'].iloc[i*block_size: (i+1)* block_size].to_list()

            inputs = tokenizer(headlines, padding = True, truncation = True, return_tensors='pt')
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

            positive = predictions[:, 0].tolist()
            negative = predictions[:, 1].tolist()
            if len(predictions[0]) < 3:
                neutral = 0
            else:
                neutral = predictions[:, 2].tolist()

            table = {"Headline":headlines,
                "Datetime": dates,
                "Positive":positive,
                "Negative":negative, 
                "Neutral":neutral
                }

            df = pd.DataFrame(table, columns = ["Headline", "Datetime", "Positive", "Negative", "Neutral"])
            df_total = df_total.append(df, ignore_index = True)
            print("Dataframe Length")
            print(len(df_total))

    return df_total


In [ ]:
#tickers = [None, 'AAPL', 'Finance', 'Market']
tickers = ['TSLA']
#models = ['bert-base-cased', 'ProsusAI/finbert']
models = ['ProsusAI/finbert']
start = "2021-04-09"
end = "2023-03-31"

for ticker in tickers:
    for model in models:
        print("-"*60)
        print("Loading " + str(ticker) + " with " + model + ".")
        print("-"*60)
        data = classifyData(start, end, ticker, model)
        if not ticker is None:
            path_end = '/' + ticker
        else:
            path_end = ''

        if not os.path.exists(r'data/sentiments/' + model + path_end):
                os.makedirs(r'data/sentiments/' + model + path_end)
        
        data.to_csv(r'data/sentiments/' + model + path_end + '/Sentiment_'+ start + '-' + end + '.csv')
